## Dependencies

In [1]:
import gc
import re
import warnings
from joblib import load
import tensorflow_hub as hub
from tensorflow.keras import Model, optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Concatenate, BatchNormalization, Activation, Average, Add, Reshape
from tensorflow.keras.layers import GlobalAveragePooling1D, Embedding, LSTM, Conv1D, SpatialDropout1D, Bidirectional, Flatten
from googleqa_utilityscript import *


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
module_url = '/kaggle/input/universalsentenceencodermodels/universal-sentence-encoder-models/use'
encoder_path = '/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'
model_path = '/kaggle/input/87-googleq-a-train-glove-text-features/model.h5'
tokenizer_path = '/kaggle/input/87-googleq-a-train-glove-text-features/tokenizer.joblib'
category_encoder_path = '/kaggle/input/87-googleq-a-train-glove-text-features/category_encoder.joblib'
host_encoder_path = '/kaggle/input/87-googleq-a-train-glove-text-features/host_encoder.joblib'
scaler_path = '/kaggle/input/87-googleq-a-train-glove-text-features/scaler.joblib'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())

    
# Text length
test['question_title_len'] = test['question_title'].apply(lambda x : len(x))
test['question_body_len'] = test['question_body'].apply(lambda x : len(x))
test['answer_len'] = test['answer'].apply(lambda x : len(x))

# Word count
test['question_title_word_cnt'] = test['question_title'].apply(lambda x : len(x.split(' ')))
test['question_body_word_cnt'] = test['question_body'].apply(lambda x : len(x.split(' ')))
test['answer_word_cnt'] = test['answer'].apply(lambda x : len(x.split(' ')))

# Unique word count
test['question_title_unique_word_cnt'] = test['question_title'].apply(lambda x : len(set(x.split(' '))))
test['question_body_unique_word_cnt'] = test['question_body'].apply(lambda x : len(set(x.split(' '))))
test['answer_unique_word_cnt'] = test['answer'].apply(lambda x : len(set(x.split(' '))))

# Misspelings count
misspelling_mapping_set = set(misspelling_mapping.keys())
test['question_title_misspelings_cnt'] = test['question_title'].apply(lambda x : len(set(x.split(' ')) & misspelling_mapping_set))
test['question_body_misspelings_cnt'] = test['question_body'].apply(lambda x : len(set(x.split(' ')) & misspelling_mapping_set))
test['answer_misspelings_cnt'] = test['answer'].apply(lambda x : len(set(x.split(' ')) & misspelling_mapping_set))

for feature in text_features:
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

for feature in text_features:
    test[feature + '_uncased'] = test[feature]

# Model parameters

In [5]:
EMBEDDDING_SIZE = 512
N_CLASS = len(target_cols)
MAX_FEATURES = 100000
TITLE_MAX_LEN = 30
BODY_MAX_LEN = 100
ANSWER_MAX_LEN = 100

## Test set

In [6]:
tokenizer = load(tokenizer_path)
ohe_category = load(category_encoder_path)
ohe_host = load(host_encoder_path)
scaler = load(scaler_path)

# Test features
# X_test_title = test['question_title']
# X_test_body = test['question_body']
# X_test_answer = test['answer']
X_test_title_seq = test['question_title_uncased']
X_test_body_seq = test['question_body_uncased']
X_test_answer_seq = test['answer_uncased']
X_test_category = ohe_category.transform(test[['category']]).toarray()
X_test_host = ohe_host.transform(test[['host']]).toarray()
X_test_dense = test[['question_title_len', 'question_title_word_cnt', 'question_title_unique_word_cnt', 'question_title_misspelings_cnt', 
                      'question_body_len', 'question_body_word_cnt', 'question_body_unique_word_cnt', 'question_body_misspelings_cnt', 
                      'answer_len', 'answer_word_cnt', 'answer_unique_word_cnt', 'answer_misspelings_cnt']]

# Tokenize the sentences
X_test_title_seq = tokenizer.texts_to_sequences(X_test_title_seq)
X_test_body_seq = tokenizer.texts_to_sequences(X_test_body_seq)
X_test_answer_seq = tokenizer.texts_to_sequences(X_test_answer_seq)

# Pad the sentences
X_test_title_seq = pad_sequences(X_test_title_seq, maxlen=TITLE_MAX_LEN)
X_test_body_seq = pad_sequences(X_test_body_seq, maxlen=BODY_MAX_LEN)
X_test_answer_seq = pad_sequences(X_test_answer_seq, maxlen=ANSWER_MAX_LEN)

X_test_dense = scaler.transform(X_test_dense)

X_test = [X_test_title_seq, X_test_body_seq, X_test_answer_seq, X_test_category, X_test_host, X_test_dense]

# Model

In [7]:
# use_embed = hub.load(module_url)

# def USEEmbedding(x):
#     return use_embed(tf.squeeze(tf.cast(x, tf.string)))

# def encoder_block(input_layer):
#     encoder = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_layer)
    
#     return encoder

## Load embedding

In [8]:
glove_embedding_matrix, glove_unknown_words = build_matrix(tokenizer.word_index, encoder_path, MAX_FEATURES)
embedding_name = 'GloVe'
print('n unknown words "%s": %d' % (embedding_name, len(glove_unknown_words)))
print('"%s" embedding shape: %s' % (embedding_name, glove_embedding_matrix.shape))

n unknown words "GloVe": 19913
"GloVe" embedding shape: (58007, 300)


In [9]:
# Title sequence
input_title_seq = Input(shape=(TITLE_MAX_LEN,), dtype=tf.float32, name='input_title_seq')
title = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_title_seq)
title = SpatialDropout1D(0.3)(title)
title = Bidirectional(LSTM(128, kernel_initializer='glorot_uniform'))(title)
title = Dropout(0.5)(title)
title = Dense(512, kernel_initializer='glorot_uniform')(title)
title = BatchNormalization()(title)
title_out = Activation('relu')(title)

# Body sequence
input_body_seq = Input(shape=(BODY_MAX_LEN,), dtype=tf.float32, name='input_body_seq')
body = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_body_seq)
body = SpatialDropout1D(0.3)(body)
body = Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='glorot_uniform'))(body)
body = Bidirectional(LSTM(128, kernel_initializer='glorot_uniform'))(body)
body = Dropout(0.5)(body)
body = Dense(512, kernel_initializer='glorot_uniform')(body)
body = BatchNormalization()(body)
body_out = Activation('relu')(body)

# Answer sequence
input_answer_seq = Input(shape=(ANSWER_MAX_LEN,), dtype=tf.float32, name='input_answer_seq')
answer = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_answer_seq)
answer = SpatialDropout1D(0.3)(answer)
answer = Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='glorot_uniform'))(answer)
answer = Bidirectional(LSTM(128, kernel_initializer='glorot_uniform'))(answer)
answer = Dropout(0.5)(answer)
answer = Dense(512, kernel_initializer='glorot_uniform')(answer)
answer = BatchNormalization()(answer)
answer_out = Activation('relu')(answer)

# Category
input_category = Input(shape=(X_test_category.shape[1],), dtype=tf.float32, name='input_category')
category = Embedding(X_test_category.shape[1], X_test_category.shape[1]//2)(input_category)
category_out = GlobalAveragePooling1D()(category)

# Host
input_host = Input(shape=(X_test_host.shape[1],), dtype=tf.float32, name='input_host')
host = Embedding(X_test_host.shape[1], X_test_host.shape[1]//2)(input_host)
host_out = GlobalAveragePooling1D()(host)

# Dense features
input_dense = Input(shape=(X_test_dense.shape[1],), dtype=tf.float32, name='input_dense')

# Output
output = Concatenate()([title_out, body_out, answer_out, category_out, host_out, input_dense])
output = Dropout(0.5)(output)
output = Dense(512, kernel_initializer='glorot_uniform')(output)
output = BatchNormalization()(output)
output = Activation('relu')(output)
output = Dropout(0.5)(output)
output = Dense(512, kernel_initializer='glorot_uniform')(output)
output = BatchNormalization()(output)
output = Activation('relu')(output)
output = Dense(N_CLASS, activation='sigmoid', kernel_initializer='glorot_uniform')(output)

model = Model(inputs=[input_title_seq, input_body_seq, input_answer_seq, input_category, input_host, input_dense], outputs=[output])

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_body_seq (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_answer_seq (InputLayer)   [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_title_seq (InputLayer)    [(None, 30)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     17402100    input_body_seq[0][0]             
______________________________________________________________________________________________

# Make predictions

In [10]:
Y_test = model.predict(X_test)

In [11]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.908490,0.617029,0.113244,0.675942,0.747020,0.634643,0.630285,0.624712,0.376539,...,0.886050,0.859956,0.586515,0.933634,0.929240,0.773916,0.071512,0.071546,0.567471,0.908718
1,46,0.871616,0.526296,0.003786,0.723159,0.789829,0.907303,0.530784,0.434676,0.073135,...,0.703583,0.941936,0.638682,0.967059,0.974738,0.860408,0.892286,0.127254,0.116911,0.886615
2,70,0.899848,0.694242,0.016798,0.818269,0.948736,0.940643,0.598449,0.574368,0.253072,...,0.880308,0.853505,0.614589,0.925721,0.954789,0.825167,0.037622,0.052760,0.755550,0.904763
3,132,0.868438,0.456070,0.005385,0.764663,0.713374,0.916591,0.513528,0.437314,0.136946,...,0.696268,0.940216,0.671586,0.968863,0.982961,0.873857,0.826505,0.152999,0.624392,0.906388
4,200,0.903590,0.490851,0.007054,0.736162,0.820339,0.828795,0.582529,0.502065,0.372090,...,0.785636,0.924203,0.623376,0.972043,0.963848,0.815609,0.456563,0.121806,0.562977,0.923146


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.885373,0.569926,0.030775,0.722876,0.815416,0.859890,0.569156,0.485842,0.221422,...,0.780213,0.929282,0.658011,0.961902,0.970840,0.860728,0.534722,0.136319,0.501192,0.907839
std,2812.670060,0.040970,0.131003,0.067394,0.094889,0.093103,0.116145,0.041096,0.074603,0.119542,...,0.087899,0.022106,0.032793,0.011361,0.013071,0.031844,0.326695,0.062924,0.263932,0.018190
min,39.000000,0.750494,0.355143,0.000923,0.333421,0.369784,0.188278,0.484993,0.372393,0.021044,...,0.567870,0.824568,0.551125,0.905766,0.870160,0.712227,0.003138,0.006475,0.056754,0.839226
25%,2572.000000,0.858410,0.450760,0.003498,0.666611,0.773487,0.856209,0.534372,0.428195,0.128628,...,0.704781,0.919217,0.636943,0.957428,0.964779,0.843634,0.188595,0.096865,0.255344,0.894066
50%,5093.000000,0.883929,0.553437,0.006335,0.733136,0.817631,0.901149,0.563502,0.456633,0.204609,...,0.782963,0.934325,0.655966,0.963633,0.973539,0.866494,0.659138,0.131657,0.505061,0.907453
75%,7482.000000,0.916972,0.677804,0.023936,0.777989,0.876704,0.922001,0.599332,0.532985,0.301270,...,0.857763,0.943962,0.681622,0.969149,0.980133,0.882616,0.830804,0.182945,0.726710,0.921273
max,9640.000000,0.968904,0.877204,0.518341,0.934234,0.986072,0.976851,0.699027,0.731780,0.625333,...,0.944032,0.965423,0.750893,0.984309,0.989759,0.925195,0.946054,0.335042,0.985162,0.947176
